In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import io # Input/Output Module
import os # OS interfaces
import cv2 # OpenCV package
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from urllib import request # module for opening HTTP requests
from matplotlib import pyplot as plt # Plotting library
from sklearn.decomposition import PCA
from typing import List, Tuple
from skimage.feature import hog
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import TSNE
import matplotlib.patches as mpatches

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset


In [ ]:
# Input data files are available in the read-only "../input/" directory

# train = pd.read_csv(
#     '/kaggle/input/kul-computer-vision-ga-1-2025/train_set.csv', index_col = 0)
# train.index = train.index.rename('id')

# test = pd.read_csv(
#     '/kaggle/input/kul-computer-vision-ga-1-2025/test_set.csv', index_col = 0)
# test.index = test.index.rename('id')

train = pd.read_csv(
    'Dataset/train_set.csv', index_col = 0)
train.index = train.index.rename('id')

test = pd.read_csv(
    'Dataset/test_set.csv', index_col = 0)
test.index = test.index.rename('id')

# read the images as numpy arrays and store in "img" column
# train['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-computer-vision-ga-1-2025/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
#                 for index, row in train.iterrows()]

# test['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-computer-vision-ga-1-2025/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
#                 for index, row in test.iterrows()]
  
train['img'] = [cv2.cvtColor(np.load('Dataset/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in train.iterrows()]

test['img'] = [cv2.cvtColor(np.load('Dataset/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in test.iterrows()]
  

train_size, test_size = len(train),len(test)

"The training set contains {} examples, the test set contains {} examples.".format(train_size, test_size)

*Note: this dataset is a subset of the* [*VGG face dataset*](https://www.robots.ox.ac.uk/~vgg/data/vgg_face/).

## 0.2. A first look
Let's have a look at the data columns and class distribution.

In [ ]:
# The training set contains an identifier, name, image information and class label
train.head(1)

In [ ]:
# The test set only contains an identifier and corresponding image information.

test.head(1)

In [ ]:
# The class distribution in the training set:
train.groupby('name').agg({'img':'count', 'class': 'max'})

Note that **Jesse is assigned the classification label 1**, and **Mila is assigned the classification label 2**. The dataset also contains 20 images of **look alikes (assigned classification label 0)** and the raw images. 

## 0.3. Preprocess data
### 0.3.1 Example: HAAR face detector
In this example we use the [HAAR feature based cascade classifiers](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html) to detect faces, then the faces are resized so that they all have the same shape. If there are multiple faces in an image, we only take the first one. 

<div class="alert alert-block alert-info"> <b>NOTE:</b> You can write temporary files to <code>/kaggle/temp/</code> or <code>../../tmp</code>, but they won't be saved outside of the current session
</div>


In [ ]:
class HAARPreprocessor():
    """Preprocessing pipeline built around HAAR feature based cascade classifiers. """
    
    def __init__(self, path, face_size):
        self.face_size = face_size
        file_path = os.path.join(path, "haarcascade_frontalface_default.xml")
        if not os.path.exists(file_path): 
            if not os.path.exists(path):
                os.mkdir(path)
            self.download_model(file_path)
        
        self.classifier = cv2.CascadeClassifier(file_path)
  
    def download_model(self, path):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/data/"\
            "haarcascades/haarcascade_frontalface_default.xml"
        
        with request.urlopen(url) as r, open(path, 'wb') as f:
            f.write(r.read())
            
    def detect_faces(self, img):
        """Detect all faces in an image."""
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return self.classifier.detectMultiScale(
            img_gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
    def extract_faces(self, img):
        """Returns all faces (cropped) in an image."""
        
        faces = self.detect_faces(img)

        return [img[y:y+h, x:x+w] for (x, y, w, h) in faces]
    
    @staticmethod
    def normalize_size(faces: List[np.ndarray]) -> Tuple[List[np.ndarray], Tuple[int, int]]:
        valid_faces = []
        for face in faces:
            if face.size == 0:
                continue
            if face.dtype != np.uint8:
                if face.dtype == np.float32 or face.dtype == np.float64:
                    face = (face * 255).astype(np.uint8)
                else:
                    face = cv2.normalize(face, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            valid_faces.append(face)
        if not valid_faces:
            return [], (0, 0)
        
        min_height = min(face.shape[0] for face in faces if face.size > 0)
        min_width = min(face.shape[1] for face in faces if face.size > 0)
        std_faces = [cv2.resize(face, (min_width, min_height)) for face in valid_faces]
        return std_faces, (min_height, min_width)
    
    def preprocess(self, data_row):
        faces = self.extract_faces(data_row['img'])
        
        # if no faces were found, return None
        if len(faces) == 0:
            nan_img = np.empty(self.face_size + (3,))
            nan_img[:] = np.nan
            return nan_img
        # only return the first face
        return cv2.resize(faces[0], self.face_size, interpolation = cv2.INTER_AREA)
            
    def __call__(self, data):
        return np.stack([self.preprocess(row) for _, row in data.iterrows()]).astype(int)

**Visualise**

Let's plot a few examples.

In [ ]:
# parameter to play with 
FACE_SIZE = (100, 100)

def plot_image_sequence(data, n, imgs_per_row=7):
    n_rows = 1 + int(n/(imgs_per_row+1))
    n_cols = min(imgs_per_row, n)

    f,ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
    for i in range(n):
        if n == 1:
            ax.imshow(data[i])
        elif n_rows > 1:
            ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(data[i])
        else:
            ax[int(i%n)].imshow(data[i])
    plt.show()

    
#preprocessed data 
preprocessor = HAARPreprocessor(path = '../../tmp', face_size=FACE_SIZE)

data_X, data_y = preprocessor(train), train['class'].values
test_X = preprocessor(test)



In [ ]:
def show_faces(faces: List[np.array]):
    """"
    Another function for visualization.
    """
    images_per_row = 5
    num_images = len(faces)
    num_rows = num_images // images_per_row + 1

    plt.figure(figsize=(15, 3 * num_rows))

    for i, img in enumerate(faces):
        plt.subplot(num_rows, images_per_row, i + 1) 
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Image {i + 1}")

    plt.tight_layout()
    plt.show()

In [ ]:
# plot faces of Michael and Sarah

plot_image_sequence(data_X[data_y == 0], n=20, imgs_per_row=10)

In [ ]:
# plot faces of Jesse

plot_image_sequence(data_X[data_y == 1], n=30, imgs_per_row=10)

In [ ]:
# plot faces of Mila

plot_image_sequence(data_X[data_y == 2], n=30, imgs_per_row=10)

## 0.4. Store Preprocessed data (optional)
<div class="alert alert-block alert-info">
<b>NOTE:</b> You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All". Feel free to use this to store intermediary results.
</div>

In [ ]:
# save preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
    
# np.save(os.path.join(prep_path, 'train_X.npy'), train_X)
# np.save(os.path.join(prep_path, 'train_y.npy'), train_y)
# np.save(os.path.join(prep_path, 'test_X.npy'), test_X)

# load preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
# train_X = np.load(os.path.join(prep_path, 'train_X.npy'))
# train_y = np.load(os.path.join(prep_path, 'train_y.npy'))
# test_X = np.load(os.path.join(prep_path, 'test_X.npy'))

Now we are ready to rock!

## 0.5 Validation Dataset Split

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets using stratified sampling
train_X, validation_X, train_y, validation_y = train_test_split(
    data_X, data_y, test_size=0.2, random_state=42, stratify=data_y
)

print(f"Training set size: {len(train_X)}")
print(f"Validation set size: {len(validation_X)}")

# Show class distribution
train_classes, train_counts = np.unique(train_y, return_counts=True)
val_classes, val_counts = np.unique(validation_y, return_counts=True)

print("\nTraining set class distribution:")
print("Class\tCount\tPercentage")
for cls, count in zip(train_classes, train_counts):
    print(f"{cls}\t{count}\t{count/len(train_y)*100:.1f}%")

print("\nValidation set class distribution:")
print("Class\tCount\tPercentage")
for cls, count in zip(val_classes, val_counts):
    print(f"{cls}\t{count}\t{count/len(validation_y)*100:.1f}%")

# 1. Feature Representations
## 1.0. Example: Identify feature extractor
Our example feature extractor doesn't actually do anything... It just returns the input:
$$
\forall x : f(x) = x.
$$

It does make for a good placeholder and baseclass ;).

In [ ]:
class IdentityFeatureExtractor:
    """A simple function that returns the input"""
    
    def transform(self, X):
        return X
    
    def __call__(self, X):
        return self.transform(X)

## 1.1. Baseline 1: HOG feature extractor
...

In [ ]:
class HOGFeatureExtractor:
    """TODO: this feature extractor is under construction"""
    def __init__(self, **params):
        self.params = {
            'orientations': 8,
            'pixels_per_cell': (4, 4),
            'cells_per_block': (2, 2),
            'block_norm': 'L2-Hys'
        }
        self.params.update(params)
        
    def transform(self, X: List[np.ndarray]) -> np.ndarray:
        """Calculate features using Histogram of Oriented Gradients and its visualisation"""
        hog_features = []
        for img in X:
            if img.dtype != np.uint8:
                img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            
            if len(img.shape) == 3:
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray_img = img
            feat = hog(gray_img, 
                      orientations=self.params['orientations'],
                      pixels_per_cell=self.params['pixels_per_cell'],
                      cells_per_block=self.params['cells_per_block'],
                      block_norm=self.params['block_norm'],
                      visualize=False)
            hog_features.append(feat)
        return np.array(hog_features)   
    
    def visualize(self, X: List[np.ndarray]) -> List[np.ndarray]:
        """Generate HOG visualizations"""
        images_per_row: int = 5
        hog_images = []
        for img in X:
            if len(img.shape) == 3:
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray_img = img
            _, hog_img = hog(gray_img, 
                           orientations=self.params['orientations'],
                           pixels_per_cell=self.params['pixels_per_cell'],
                           cells_per_block=self.params['cells_per_block'],
                           block_norm=self.params['block_norm'],
                           visualize=True)
            hog_images.append(hog_img)

        n_images = len(hog_images)
        n_rows = (n_images + images_per_row - 1) // images_per_row
        plt.figure(figsize=(15, 3 * n_rows))
        
        for i, hog_img in enumerate(hog_images):
            plt.subplot(n_rows, images_per_row, i + 1)
            plt.imshow(hog_img, cmap='gray')
            plt.axis('off')
            plt.title(f"HOG {i+1}")
        
        plt.tight_layout()
        plt.show()
    

In [ ]:
preprocessor = HAARPreprocessor(path='../../tmp', face_size=FACE_SIZE)
data_X = preprocessor(train)

hog_extractor = HOGFeatureExtractor(orientations=9, pixels_per_cell=(16, 16))
normalized_faces, _ = preprocessor.normalize_size(data_X)
hog_features = hog_extractor.transform(normalized_faces)
hog_images = hog_extractor.visualize(normalized_faces)

### 1.1. Alternative Baseline 2: Scale Invariant Feature Transform


In [ ]:
class SIFTFeatureExtractor:
    """Feature extraction using Scale Invariant Feature Transform (SIFT)"""
    def __init__(self, **params):
        self.params = params
        self.sift = cv2.SIFT_create(**params)
        
    def extract_descriptors(self, X: List[np.ndarray]) -> List[np.ndarray]:
        """Calculate features using Scale Invariant Feature Transform"""
        descriptors_list = []
        for img in X:
            if img.dtype != np.uint8:
                img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            if len(img.shape) == 3:
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray_img = img
            _, descriptors = self.sift.detectAndCompute(gray_img, None)
            if descriptors is None:
                descriptors = np.zeros((0, 128), dtype=np.float32)
            descriptors_list.append(descriptors)
        return descriptors_list

    def visualize_keypoints(self, X: List[np.ndarray], images_per_row: int = 5):
        """Visualize SIFT keypoints on images"""
        keypoints_list = []
        for img in X:
            if len(img.shape) == 3:
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray_img = img
            keypoints, _ = self.sift.detectAndCompute(gray_img, None)
            keypoints_list.append(keypoints)
        
        num_images = len(X)
        num_rows = (num_images + images_per_row - 1) // images_per_row
        plt.figure(figsize=(15, 3 * num_rows))
        
        for i, (img, kps) in enumerate(zip(X, keypoints_list)):
            plt.subplot(num_rows, images_per_row, i + 1)
            img_with_kps = cv2.drawKeypoints(img, kps, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            img_with_kps = cv2.cvtColor(img_with_kps, cv2.COLOR_BGR2RGB)
            plt.imshow(img_with_kps)
            plt.axis('off')
            plt.title(f"SIFT Keypoints {i+1}")
        plt.tight_layout()
        plt.show()


In [ ]:
sift_extractor = SIFTFeatureExtractor()
descriptors_list = sift_extractor.extract_descriptors(normalized_faces)
sift_extractor.visualize_keypoints(normalized_faces)

### 1.1.1. t-SNE Plots
...

In [ ]:
class FeatureVisualizer:    
    @staticmethod
    def plot_tsne(features: np.ndarray, labels: np.ndarray, title: str):
        """Data visualisation using T-sne"""
        tsne = TSNE(n_components=2, perplexity=15, random_state=123)
        tsne_2d = tsne.fit_transform(features)
        
        plt.figure(figsize=(8, 6))
        scatter = plt.scatter(tsne_2d[:, 0], tsne_2d[:, 1], c=labels, cmap='tab10')
        unique_labels = np.unique(labels)
        handles = [mpatches.Patch(color=scatter.cmap(scatter.norm(label)), label=str(label)) 
                  for label in unique_labels]
        plt.legend(handles=handles, title="Class")
        plt.title(title)
        plt.show()

    @staticmethod
    def build_codebook(descriptors_list: List[np.ndarray], n_clusters: int = 128):
        """Combine descriptors and create codebook for Bag-of-Words approach"""
        all_descriptors = np.vstack([desc for desc in descriptors_list if desc.size > 0])
        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=123)
        kmeans.fit(all_descriptors)
        return kmeans

    @staticmethod
    def compute_bow_histograms(descriptors_list: List[np.ndarray], kmeans) -> np.ndarray:
        """Compute Bag-of-Words histograms"""
        n_clusters = kmeans.n_clusters
        histograms = []
        for desc in descriptors_list:
            if desc.size == 0:
                hist = np.zeros(n_clusters)
            else:
                words = kmeans.predict(desc)
                hist, _ = np.histogram(words, bins=np.arange(n_clusters + 1))
                hist = hist.astype(float)
                if hist.sum() > 0:
                    hist /= hist.sum()
            histograms.append(hist)
        return np.array(histograms)

In [ ]:
visualizer = FeatureVisualizer()
labels = train['class'].to_numpy()
visualizer.plot_tsne(hog_features, labels, "t-SNE of HOG Features")

In [ ]:
kmeans = visualizer.build_codebook(descriptors_list, n_clusters=128)
sift_histograms = visualizer.compute_bow_histograms(descriptors_list, kmeans)
visualizer.plot_tsne(sift_histograms, labels, "t-SNE of SIFT Features")

### 1.1.2. Discussion
...

## 1.2. Baseline 2: PCA feature extractor
...

In [ ]:
def plot_variance_explained(faces: List[np.ndarray], whiten: bool) -> None:
    """
    Plot variance - number of PCs.
    """
    face_data = np.array([face.flatten() for face in faces])
    max_components = min(face_data.shape[0], face_data.shape[1])
    pca = PCA(n_components=max_components, whiten=whiten)
    pca.fit(face_data)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    
    plt.figure(figsize=(8, 6))
    plt.plot(np.arange(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-')
    plt.xlabel("Number of Principal Components")
    plt.ylabel("Cumulative Explained Variance")
    plt.title("Explained Variance vs. Number of Principal Components")
    plt.grid(True)
    plt.show()

In [ ]:
class PCAFeatureExtractor(IdentityFeatureExtractor):
    """TODO: this feature extractor is under construction"""
    
    def __init__(self, n_components: int, whiten: bool):
        self.pca = PCA(n_components=n_components, whiten=whiten)
        self.mean_face = None
        self.eigenvectors = None

    def fit(self, faces: List[np.array]):
        """
        Performs PCA give the number of principle components.
        """
        face_data = np.array([face.flatten() for face in faces])
        self.pca.fit(face_data)
        self.mean_face = self.pca.mean_
        self.eigenvectors = self.pca.components_

    def transform(self, faces: List[np.array]) -> List[np.array]:
        """
        Transform original face images to the PCA space.
        """
        face_data = np.array([face.flatten() for face in faces])
        projections = self.pca.transform(face_data)
        return projections

    def fit_transform(self, faces: List[np.array]) -> List[np.array]:
        """
        Fit PCA on the data at first then transform the faces to the PCA space.
        """
        self.fit(faces)
        projections = self.transform(faces)
        return projections
    
    def inverse_transform(self, projections: np.array) -> List[np.array]:
        """
        Convert projections to the original image space.
        """
        reconstructed = np.dot(projections, self.eigenvectors) + self.mean_face
        return reconstructed

In [ ]:
def vectors_to_images(vectors: List[np.array], std_shape: tuple) -> List[np.array]:
    """
    Change 1D vectors to 2D images for visualization.

    std_shape: tuple(H, W, C)
        Shape of the target image.
    """
    images = []
    for vector in vectors:
        image = vector.reshape(std_shape[0], std_shape[1], std_shape[2])
        image_shifted = image - np.min(image)
        image_scaled = 255 * (image_shifted / np.max(image_shifted))
        image_display = np.round(image_scaled).astype(np.uint8)
        images.append(image_display)
    return images

In [ ]:
std_shape = (train_X.shape[1], train_X.shape[2], 3)

In [ ]:
plot_variance_explained(train_X, False)

In [ ]:
pca = PCAFeatureExtractor(50, False)
projections = pca.fit_transform(train_X)
reconstructed_vectors = pca.inverse_transform(projections)

### 1.2.1. Eigenface Plots
...

In [ ]:
eigenfaces = vectors_to_images(pca.eigenvectors, std_shape)
show_faces(eigenfaces)

### 1.2.2. Feature Space Plots
...

In [ ]:
reconstructed_faces = vectors_to_images(reconstructed_vectors, std_shape)
show_faces(reconstructed_faces)

### 1.2.3. Discussion
...

# 2. Evaluation Metrics
## 2.0. Example: Accuracy
As example metric we take the accuracy. Informally, accuracy is the proportion of correct predictions over the total amount of predictions. It is used a lot in classification but it certainly has its disadvantages...

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(true_labels, predictions, class_names=None, model_name="Model"):
    """
    Evaluate classification performance and plot confusion matrix.

    Parameters:
    - true_labels: ground truth labels
    - predictions: model predicted labels
    - class_names: list of class names (optional)
    - model_name: name for labeling the confusion matrix plot
    """
    # Accuracy
    accuracy = accuracy_score(true_labels, predictions)
    print(f"\n[{model_name}] Accuracy: {accuracy:.4f}")

    # Classification Report
    print(f"\n[{model_name}] Classification Report:\n", classification_report(true_labels, predictions))

    # Confusion Matrix
    cm = confusion_matrix(true_labels, predictions)
    print(f"[{model_name}] Confusion Matrix:\n", cm)

    # Plot Confusion Matrix Heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names if class_names else sorted(set(true_labels)),
                yticklabels=class_names if class_names else sorted(set(true_labels)))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.show()


# 3. Classifiers
## 3.0. Example: The *'not so smart'* classifier
This random classifier is not very complicated. It makes predictions at random, based on the distribution obseved in the training set. **It thus assumes** that the class labels of the test set will be distributed similarly to the training set.

In [ ]:
class RandomClassificationModel:
    """Random classifier, draws a random sample based on class distribution observed 
    during training."""
    
    def fit(self, X, y):
        """Adjusts the class ratio instance variable to the one observed in y. 

        Parameters
        ----------
        X : tensor
            Training set
        y : array
            Training set labels

        Returns
        -------
        self : RandomClassificationModel
        """
        
        self.classes, self.class_ratio = np.unique(y, return_counts=True)
        self.class_ratio = self.class_ratio / self.class_ratio.sum()
        return self
        
    def predict(self, X):
        """Samples labels for the input data. 

        Parameters
        ----------
        X : tensor
            dataset
            
        Returns
        -------
        y_star : array
            'Predicted' labels
        """

        np.random.seed(0)
        return np.random.choice(self.classes, size = X.shape[0], p=self.class_ratio)
    
    def __call__(self, X):
        return self.predict(X)
    

## 3.1. Decision Tree

In [ ]:
class DecisionTreeModel:
    def __init__(self, max_depth: int = None, random_state: int = None):
        """
        Initialization.
        """
        self.max_depth = max_depth
        self.random_state = random_state
        self.model = DecisionTreeClassifier(max_depth=self.max_depth,
                                            random_state=self.random_state)
    
    def fit(self, faces: np.array, labels: np.array):
        """
        Fit the Decision Tree model.

        Parameters:
        - faces: 
            2D np.array or tensor with shape [n_samples, n_features]. Each face should be represented as a 1D vector.
        - labels:
            1D np.array containing the class labels.
        """
        faces = np.array([face.flatten() for face in faces])
        self.model.fit(faces, labels)
    
    def predict(self, faces: np.array) -> np.array:
        """
        Predict class for faces using the trained Decision Tree model.

        Parameters:
        - faces:
            2D np.array or tensor with shape [n_samples, n_features]. Each face should be represented as a 1D vector.

        Returns:
        - np.array: The predicted labels for all input faces.
        """
        faces = np.array([face.flatten() for face in faces])
        return self.model.predict(faces)

## 3.2. RandomForest

In [ ]:
class RandomForestModel:
    def __init__(self, n_estimators: int = 100, max_depth: int = None, random_state: int = None):
        """
        Initialization
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.model = RandomForestClassifier(n_estimators=self.n_estimators,
                                            max_depth=self.max_depth,
                                            random_state=self.random_state)
    
    def fit(self, faces: np.array, labels: np.array):
        """
        Fit the model.

        Parameters:
        faces: 
            2d np.array or tensor with shape [n_sample, n_feature], namely each face should be
            represented with 1D vectors
        labels:
            1d np.array containing labels.
        """
        faces = np.array([face.flatten() for face in faces])
        self.model.fit(faces, labels)
    
    def predict(self, faces: np.array) -> np.array:
        """
        Predict class for faces.

        Parameters:
        faces: 
            2d np.array or tensor with shape [n_sample, n_feature], namely each face should be
            represented with 1D vectors

        Return:
            The predicted labels of all the input faces.
        """
        faces = np.array([face.flatten() for face in faces])
        return self.model.predict(faces)

## 3.3. Support Vector Machine (SVM)

In [ ]:
class SVMClassifier:
    """Classifier using Support Vector Machine (SVM) algorithm"""
    
    def __init__(self, **params):
        """
        Initialize SVM classifier with given parameters.
        
        Parameters:
        -----------
        C: float (default=1.0) - regulatisation parameter
        kernel: str (default='rbf') - specify the kernel type
        gamma: str or float (default='scale') 
        random_state: int (default='42')
        and more
        """
        default_params = {
            'C': 1.0,
            'kernel': 'rbf',
            'gamma': 'scale',
            'random_state': 42
        }
        default_params.update(params)
        self.model = SVC(**default_params)
        self.scaler = StandardScaler()
        self.is_trained = False
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Train the SVM classifier on the given data.
        
        Parameters:
        -----------
        X : np.ndarray
            Feature matrix of shape (n_samples, n_features)
        y : np.ndarray
            Target labels of shape (n_samples,)
        """
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)
        self.is_trained = True
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict class labels for samples using trained model

        Parameters:
        -----------
        X: np.ndarray
            Feature matrix of shape(face) (n_samples, n_features)
        
        Returns: 
        --------
        np.ndarray - predicted class label of faces
        """
        if not self.is_trained:
            raise RuntimeError("Model has not been trained yet. Call fit() first.")  
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)

## 3.4. CNN (ImageNet Based)

In [ ]:
class CNN:
    """
    A CNN based on mobilenet_v2 model.
    """
    def __init__(self, num_classes=3, lr=0.001, num_epochs=10, batch_size=16):
        self.num_classes = num_classes
        self.lr = lr
        self.num_epochs = num_epochs
        self.batch_size = batch_size

        self.model = models.mobilenet_v2(pretrained=True)
        
        # Freeze pretrained layers' parameters.
        for param in self.model.features.parameters():
            param.requires_grad = False
        
        # Contenate a classification head after ImageNet, only this part is trained.
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.model.last_channel, 64),
            nn.ReLU(),
            nn.Linear(64, self.num_classes)
        )
        
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        elif torch.backends.mps.is_available():
            self.device = torch.device("mps")
        else:
            self.device = torch.device("cpu")
        self.model.to(self.device)
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.is_trained = False
        
        # Normalize to the size required by mobileNet
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Train the CNN classifier.
        
        Parameters:
        X : np.ndarray
            Array of images with shape (n_samples, H, W, C).
        y : np.ndarray
            Array of target labels with shape (n_samples,).
        """
        # Load the data to Dataset for training.
        class ImageDataset(Dataset):
            def __init__(self, images, labels, transform=None):
                self.images = images
                self.labels = labels
                self.transform = transform
                
            def __len__(self):
                return len(self.images)
            
            def __getitem__(self, idx: int) -> tuple:
                """
                Output the image given the index.
                """
                image = self.images[idx]
                image = Image.fromarray(image)
                if self.transform:
                    image = self.transform(image)
                label = int(self.labels[idx])
                return image, label
        
        dataset = ImageDataset(X, y, transform=self.transform)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        self.model.train()
        for epoch in range(self.num_epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            
            for images, labels in dataloader:
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                
                running_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            
            epoch_loss = running_loss / len(dataset)
            epoch_acc = correct / total
            print(f"Epoch {epoch+1}/{self.num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")
        
        self.is_trained = True
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        if not self.is_trained:
            raise RuntimeError("Model has not been trained yet. Call fit() first.")
        
        # Create a dataset for inference.
        class PredictDataset(Dataset):
            def __init__(self, images, transform=None):
                self.images = images
                self.transform = transform
                
            def __len__(self):
                return len(self.images)
            
            def __getitem__(self, idx):
                image = self.images[idx]
                image = Image.fromarray(image)
                if self.transform:
                    image = self.transform(image)
                return image
        
        dataset = PredictDataset(X, transform=self.transform)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
        
        self.model.eval()
        all_preds = []
        with torch.no_grad():
            for images in dataloader:
                images = images.to(self.device)
                outputs = self.model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
        return np.array(all_preds)

# 4. Experiments
<div class="alert alert-block alert-info"> <b>NOTE:</b> Do <i>NOT</i> use this section to keep track of every little change you make in your code! Instead, highlight the most important findings and the major (best) pipelines that you've discovered.  
</div>
<br>

## 4.0. Example: basic pipeline
The basic pipeline takes any input and samples a label based on the class label distribution of the training set. As expected the performance is very poor, predicting approximately 1/4 correctly on the training set. There is a lot of room for improvement but this is left to you ;). 

In [ ]:
feature_extractor = IdentityFeatureExtractor() 
classifier = RandomClassificationModel()

# train the model on the features
classifier.fit(feature_extractor(train_X), train_y)

# model/final pipeline
model = lambda X: classifier(feature_extractor(X))

In [ ]:
# evaluate performance of the model on the training set
train_y_star = model(validation_X)

"The performance on the training set is {:.2f}. This however, does not tell us much about the actual performance (generalisability).".format(
    accuracy_score(validation_y, train_y_star))

In [ ]:
# predict the labels for the test set 
test_y_star = model(test_X)

## 4.1 PCA + DecisionTree

In [ ]:
pca = PCAFeatureExtractor(50, False)
projections = pca.fit_transform(train_X)
cls_dt = DecisionTreeModel()
cls_dt.fit(projections, train_y)

validation_X_pca = pca.transform(validation_X)
predictions = cls_dt.predict(validation_X_pca)
accuracy_score(validation_y, predictions)
evaluate_model(validation_y, predictions, class_names=["Michael", "Sarah", "Unknown"], model_name="PCA + DecisionTree")


## 4.2 PCA + RandomForest

In [ ]:
pca = PCAFeatureExtractor(50, False)
projections = pca.fit_transform(train_X)
cls_rf = RandomForestModel()
cls_rf.fit(projections, train_y)

validation_X_pca = pca.transform(validation_X)
predictions = cls_rf.predict(validation_X_pca)
accuracy_score(validation_y, predictions)

evaluate_model(validation_y, predictions, class_names=["Michael", "Sarah", "Unknown"], model_name="PCA + RandomForest")


## 4.3 PCA + SVM

In [ ]:
pca = PCAFeatureExtractor(50, False)
projections = pca.fit_transform(train_X)
svm = SVMClassifier(C=0.1, gamma = 0.001 , kernel='rbf')
svm.fit(projections, train_y)

validation_X_pca = pca.transform(validation_X)
predictions = svm.predict(validation_X_pca)
accuracy = accuracy_score(validation_y, predictions)
print(f"\n Accuracy: {accuracy:.4f}")

evaluate_model(validation_y, predictions, class_names=["Michael", "Sarah", "Unknown"], model_name="PCA + SVM")


## 4.4 PCA + CNN

In [ ]:
pca = PCAFeatureExtractor(50, False)
projections = pca.fit_transform(train_X)
reconstructed_vectors = pca.inverse_transform(projections)
reconstructed_faces = vectors_to_images(reconstructed_vectors, std_shape)

cnn_clf = CNN()
cnn_clf.fit(reconstructed_faces, train_y)

validation_X_pca = pca.transform(validation_X)
reconstructed_vectors = pca.inverse_transform(validation_X_pca)
reconstructed_faces = vectors_to_images(reconstructed_vectors, std_shape)
predictions = cnn_clf.predict(reconstructed_faces)
accuracy = accuracy_score(validation_y, predictions)
print(f"\n Accuracy: {accuracy:.4f}")

evaluate_model(validation_y, predictions, class_names=["Michael", "Sarah", "Unknown"], model_name="PCA + CNN")


## 4.5 HOG + SVM

In [ ]:
hog_extractor = HOGFeatureExtractor(orientations=9, pixels_per_cell=(16, 16))
X_train_hog = hog_extractor.transform(train_X)
X_val_hog = hog_extractor.transform(validation_X)
svm = SVMClassifier(C=10, kernel='rbf')
svm.fit(X_train_hog, train_y)
predictions = svm.predict(X_val_hog)
accuracy = accuracy_score(validation_y, predictions)
print(f"\n Accuracy: {accuracy:.4f}")

evaluate_model(validation_y, predictions, class_names=["Michael", "Sarah", "Unknown"], model_name="HOG + SVM")


# 5. Publishing best results

In [ ]:
submission = test.copy().drop('img', axis = 1)
submission['class'] = test_y_star

submission

In [ ]:
submission.to_csv('submission.csv')

# 6. Discussion
...

In summary we contributed the following: 
* 
